In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
from google.colab import files
src = list(files.upload().values())[0]


Saving clr_callback.py to clr_callback.py


In [ ]:
open('clr_callback.py','wb').write(src)
import clr_callback

Using TensorFlow backend.


In [ ]:
from google.colab import files
src = list(files.upload().values())[0]

Saving learningratefinder.py to learningratefinder.py


In [ ]:
open('learningratefinder.py','wb').write(src)
import learningratefinder

In [ ]:
from google.colab import files
src = list(files.upload().values())[0]

Saving config.py to config.py


In [ ]:
open('config.py','wb').write(src)
import config

In [ ]:

# set the matplotlib backend so figures can be saved in the background
from google.colab import files
import matplotlib
matplotlib.use("Agg")
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
import pandas as pd
import os
import shutil
import random
from shutil import copyfile
from clr_callback import CyclicLR
from learningratefinder import LearningRateFinder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import argparse
import cv2
import sys
import keras
print(tf.__version__)
print(keras.__version__)

1.15.2
2.3.1


In [ ]:
train_data_dir = "/content/drive/My Drive/Colab Notebooks/Classification/Skin_Cancer/training/"
validation_data_dir ="/content/drive/My Drive/Colab Notebooks/Classification/Skin_Cancer/validation/"
# build the DenseNet201 network
#include_top=False means that we want to ignore the fully connected layer on the top and get straight to the convolutions
pre_trained_model = tf.keras.applications.DenseNet201(input_shape=(150,150,3),weights='imagenet', include_top=False)
print('Model loaded.')
pre_trained_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
74842112/74836368 [==============================] - 1s 0us/step
Model loaded.
Model: "densenet201"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 75, 75, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalizatio

In [ ]:
TRAINING_DIR = "/content/drive/My Drive/Colab Notebooks/Classification/Skin_Cancer/training/"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    color_mode="rgb",
                                                    batch_size=128,
                                                    class_mode='categorical',
                                                    target_size=(150,  150))


VALIDATION_DIR = "/content/drive/My Drive/Colab Notebooks/Classification/Skin_Cancer/validation/"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              color_mode="rgb",
                                                              batch_size=128,
                                                              class_mode='categorical',
                                                              target_size=(150,150))

Found 15932 images belonging to 7 classes.
Found 1773 images belonging to 7 classes.


In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(pre_trained_model.layers))

# Freeze all the layers 
for layer in pre_trained_model.layers:
  layer.trainable =  True


last_layer = pre_trained_model.get_layer('relu')
print('last layer output shape: ', last_layer.output_shape)
last_output= last_layer.output

Number of layers in the base model:  707
last layer output shape:  (None, 4, 4, 1920)


In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (7, activation='softmax')(x)    

In [ ]:
from itertools import tee  
from sklearn import metrics

from tensorflow.keras.callbacks import *

class Metrics(Callback):
    def __init__(self, validation_generator, validation_steps, threshold=0.5):
        self.validation_generator = validation_generator
        self.validation_steps = validation_steps or len(validation_generator)
        self.threshold = threshold

    def on_train_begin(self, logs={}):
        self.val_f1_scores = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        # duplicate generator to make sure y_true and y_pred are calculated from the same observations
        gen_1 =self.validation_generator
        gen_2=self.validation_generator
        y_true = np.vstack(next(gen_1)[1] for _ in range(self.validation_steps)).astype('int')
        y_pred = (self.model.predict_generator(gen_2, steps=self.validation_steps) > self.threshold).astype('int')
        f1 = metrics.f1_score(y_true, y_pred, average='weighted')
        precision = metrics.precision_score(y_true, y_pred, average='weighted')
        recall = metrics.recall_score(y_true, y_pred, average='weighted')
        self.val_f1_scores.append(f1)
        self.val_recalls.append(recall)
        self.val_precisions.append(precision)
        print(f" - val_f1_score: {f1:.5f} - val_precision: {precision:.5f} - val_recall: {recall:.5f} ")
        return

In [ ]:
# initialize the optimizer and model
print("[INFO] compiling model...")
# add the model on top of the convolutional base
model = Model( pre_trained_model.input, x) 
optimizer = tf.keras.optimizers.Adam(lr=0.000001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.summary()
model.compile(optimizer = optimizer, 
              loss = "categorical_crossentropy")

[INFO] compiling model...
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 75, 75, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1/conv[0][0]                 
____________________________________________________________________

In [ ]:
LRFIND_PLOT_PATH = os.path.sep.join(["/content/drive/My Drive/Colab Notebooks/Classification/Skin_Cancer", "lrfind_plot.png"])
TRAINING_PLOT_PATH = os.path.sep.join(["/content/drive/My Drive/Colab Notebooks/Classification/Skin_Cancer", "training_plot.png"])
CLR_PLOT_PATH = os.path.sep.join(["/content/drive/My Drive/Colab Notebooks/Classification/Skin_Cancer", "clr_plot.png"])


In [ ]:
# check to see if we are attempting to find an optimal learning rate
# before training for the full number of epochs
# initialize the learning rate finder and then train with learning
# rates ranging from 1e-10 to 1e+1
print("[INFO] finding learning rate...")
lrf = LearningRateFinder(model)
lrf.find(train_generator,
		1e-10, 1e+1,
		stepsPerEpoch=50,
		batchSize=32)

# plot the loss for the various learning rates and save the
# resulting plot to disk
lrf.plot_loss()
plt.savefig(LRFIND_PLOT_PATH)

# gracefully exit the script so we can adjust our learning rates
# in the config and then train the network for our full set of
# epochs
print("[INFO] learning rate finder complete")
print("[INFO] examine plot and adjust learning rates before training")
sys.exit(0)

[INFO] finding learning rate...
Epoch 1/10
50/50 [==============================] - 843s 17s/step - loss: 3.5558 - precision: 0.0963 - recall: 0.0556 - auc: 0.4392
Epoch 2/10
50/50 [==============================] - 832s 17s/step - loss: 3.4969 - precision: 0.1076 - recall: 0.0575 - auc: 0.4444
Epoch 3/10
50/50 [==============================] - 836s 17s/step - loss: 3.4689 - precision: 0.1055 - recall: 0.0587 - auc: 0.4468
Epoch 4/10
50/50 [==============================] - 836s 17s/step - loss: 2.9839 - precision: 0.1649 - recall: 0.0862 - auc: 0.5205
Epoch 5/10
50/50 [==============================] - 808s 16s/step - loss: 2.0115 - precision: 0.4062 - recall: 0.2681 - auc: 0.7371
Epoch 6/10
50/50 [==============================] - 811s 16s/step - loss: 1.5225 - precision: 0.6029 - recall: 0.4650 - auc: 0.8466
Epoch 7/10
50/50 [==============================] - 786s 16s/step - loss: 2.2271 - precision: 0.6004 - recall: 0.3972 - auc: 0.8362
Epoch 8/10
50/50 [==========================

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#os.mkdir("/content/drive/My Drive/Colab Notebooks/Classification/Skin_Cancer/Checkpoints_file_8_exp_range")
checkpoint_path = "/content/drive/My Drive/Colab Notebooks/Classification/Skin_Cancer/Checkpoints_file_8_exp_range/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
# otherwise, we have already defined a learning rate space to train
# over, so compute the step size and initialize the cyclic learning
# rate method
stepSize = config.STEP_SIZE * 124
clr = CyclicLR(
	mode=config.CLR_METHOD,
	base_lr=0.000001,
	max_lr=0.001,
	step_size=stepSize)


In [ ]:
#continue training
metrics_new = Metrics(validation_generator, validation_steps=14)
initial_epochs=45
history = model.fit_generator(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch =124,
            epochs = initial_epochs,
            validation_steps =14,
            callbacks=[clr,cp_callback,metrics_new],
            class_weight={0:2.38574423,1:0.76452805,2:1.13856928,3:10.63551402,4:0.64530763,5:0.37719589,6:10.02643172},
            initial_epoch=39,
            verbose = 1 )

In [ ]:
model.load_weights("/content/drive/My Drive/Colab Notebooks/Classification/Skin_Cancer/BEST_EPOCH/cp.ckpt")

In [ ]:
gen_1 =validation_generator
gen_2=validation_generator
y_true = np.vstack(next(gen_1)[1] for _ in range(14)).astype('int')
y_pred = (model.predict_generator(gen_2, steps=14) > 0.5).astype('int')
print("The weighted f1 score is:")
f1 = metrics.f1_score(y_true, y_pred, average='weighted')
print(f1)
print("The weighted precision score is:")
precision = metrics.precision_score(y_true, y_pred, average='weighted')
print(precision)
print("The weighted recall score is:")
recall = metrics.recall_score(y_true, y_pred, average='weighted')
print(recall)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


The weighted f1 score is:
0.8488029766468725
The weighted precision score is:
0.8618051646704384
The weighted recall score is:
0.838127467569092


In [ ]:
from sklearn.metrics import balanced_accuracy_score
gen_1 =validation_generator
gen_2=validation_generator
y_true = np.vstack(next(gen_1)[1] for _ in range(14)).astype('int')
y_pred = (model.predict_generator(gen_2, steps=14) > 0.5).astype('int')
y_true=np.argmax(y_true,axis=1)
y_pred=np.argmax(y_pred,axis=1)
print("The balanced accuracy score is:")
balanced_accuracy_score(y_true, y_pred)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  after removing the cwd from sys.path.


The balanced accuracy score is:


0.8358916160811348

In [ ]:
from sklearn.metrics import roc_auc_score
gen_1 =validation_generator
gen_2=validation_generator
y_true = np.vstack(next(gen_1)[1] for _ in range(14)).astype('int')
y_pred = (model.predict_generator(gen_2, steps=14) > 0.5).astype('int')
print("The weighted roc auc score is:")
roc_auc_score(y_true, y_pred,'weighted')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  after removing the cwd from sys.path.


The weighted roc auc score is:


0.9011728895499188

In [ ]:
from sklearn.metrics import classification_report
gen_1 =validation_generator
gen_2=validation_generator
y_true = np.vstack(next(gen_1)[1] for _ in range(14)).astype('int')
y_pred = (model.predict_generator(gen_2, steps=14) > 0.5).astype('int')
y_true=np.argmax(y_true,axis=1)
y_pred=np.argmax(y_pred,axis=1)
target_names=['AKIEC','BCC','BKL','DF','MEL','NV','VASC']
print(classification_report(y_true, y_pred,target_names=target_names))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  after removing the cwd from sys.path.


              precision    recall  f1-score   support

       AKIEC       0.55      0.70      0.61       106
         BCC       0.91      0.81      0.86       331
         BKL       0.71      0.77      0.74       223
          DF       0.68      0.88      0.76        24
         MEL       0.84      0.83      0.83       392
          NV       0.93      0.91      0.92       671
        VASC       0.89      0.96      0.93        26

    accuracy                           0.84      1773
   macro avg       0.79      0.84      0.81      1773
weighted avg       0.85      0.84      0.85      1773

